<a href="https://colab.research.google.com/github/amitnits2016/Keras/blob/master/RetinaNet/Object_Detection_retinanet_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Our Google Drive should have Data Folder, resnet50_coco_best_v2.0.1.h5 and data_load.py file

#### Our data contains three objects:
1. chair 
2. table 
3. vase

#### Mount Google Drive to add Data Folder, In which Two separate folders Train and Test will be there. Each Folder is containing 3 files:
1. jpg file  (Train: 167, Test: 55)
2. xml file  (Train: 167, Test: 55)
3. text file (Train: 167, Test: 55)

#### The folders will also contain a class file where each object is labelled. (Train: 1, Test: 1)

#### This way 
#### Train Folder will have 167*3+1=502 files total
#### Test Folder will have 55*3+1=166 files total


# Cloning keras-retinanet using below commands

In [1]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 5918 (delta 19), reused 18 (delta 10), pack-reused 5885
Receiving objects: 100% (5918/5918), 13.42 MiB | 6.96 MiB/s, done.
Resolving deltas: 100% (3983/3983), done.


### Check the directory folder, and go to keras-retinanet folder

In [2]:
ls

drive/  keras-retinanet/  sample_data/


In [3]:
cd keras-retinanet/

/content/keras-retinanet


In [4]:
ls

CONTRIBUTORS.md  keras_retinanet/  requirements.txt  snapshots/
examples/        LICENSE           setup.cfg         tests/
images/          README.md         setup.py


#### Drag and Drop data_load.py file of Google Drive into keras-retinanet folder
#### Drag and Drop resnet50_coco_best_v2.0.1.h5 file of Google Drive into keras-retinanet/snapshots folder

#### Check If It is moved properly or not

In [5]:
ls

CONTRIBUTORS.md  images/           README.md         setup.py
data_load.py     keras_retinanet/  requirements.txt  snapshots/
examples/        LICENSE           setup.cfg         tests/


#### Modify the ./keras_retinanet/bin/train.py file, so that more than 5 epochs can be run without stopping.

In [6]:
%pycat ./keras_retinanet/bin/train.py

#### run the setup.py file

In [8]:
!python3 setup.py build install

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/keras_retinanet
copying keras_retinanet/losses.py -> build/lib.linux-x86_64-3.6/keras_retinanet
copying keras_retinanet/__init__.py -> build/lib.linux-x86_64-3.6/keras_retinanet
copying keras_retinanet/initializers.py -> build/lib.linux-x86_64-3.6/keras_retinanet
creating build/lib.linux-x86_64-3.6/tests
copying tests/__init__.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_losses.py -> build/lib.linux-x86_64-3.6/tests
creating build/lib.linux-x86_64-3.6/keras_retinanet/models
copying keras_retinanet/models/retinanet.py -> build/lib.linux-x86_64-3.6/keras_retinanet/models
copying keras_retinanet/models/senet.py -> build/lib.linux-x86_64-3.6/keras_retinanet/models
copying keras_retinanet/models/mobilenet.py -> build/lib.linux-x86_64-3.6/keras_retinanet/models
copying keras_retinanet/models/resnet.py -> build/lib.linux-x86_64-3.6/keras_retinanet/models
copying k

In [9]:
ls

build/           examples/                  LICENSE           setup.py
CONTRIBUTORS.md  images/                    README.md         snapshots/
data_load.py     keras_retinanet/           requirements.txt  tests/
dist/            keras_retinanet.egg-info/  setup.cfg


#### 1) loading data, by running data_load.py file with proper input Test and Train file

In [19]:
!python3 data_load.py --train '/content/drive/My Drive/RetinaNet/Data/Train/' --test '/content/drive/My Drive/RetinaNet/Data/Test/'

DONE


#### 2) Train data, by running train.py file

In [15]:
!python3 setup.py build_ext --inplace

running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils


In [17]:
!python /content/keras-retinanet/keras_retinanet/bin/train.py --backbone='resnet50' --weights /content/keras-retinanet/snapshots/resnet50_coco_best_v2.0.1.h5 --random-transform --gpu=0 --batch-size=4 --steps 600 --epochs 2 --image-min-side=500 --image-max-side=500 --lr 0.001 --weighted-average --compute-val-loss csv /content/keras-retinanet/annotations.csv /content/keras-retinanet/classes.csv \
--val-annotations /content/keras-retinanet/val_annotations.csv

Using TensorFlow backend.
2020-08-03 12:35:13.758488: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-03 12:35:15.646863: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-03 12:35:15.665790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-03 12:35:15.666542: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-03 12:35:15.666583: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-03 12:35:15.668422: I tensorflow/str

In [16]:
!python3 /content/keras-retinanet/keras_retinanet/bin/train.py --backbone='resnet50' --weights /content/keras-retinanet/snapshots/resnet50_coco_best_v2.0.1.h5 --random-transform --gpu=0 --batch-size=4 --steps 600 --epochs 12 --image-min-side=500 --image-max-side=500 --lr 0.001 --weighted-average --compute-val-loss csv /content/keras-retinanet/annotations.csv /content/keras-retinanet/classes.csv \
--val-annotations /content/keras-retinanet/val_annotations.csv

Using TensorFlow backend.
2020-08-03 12:27:57.036284: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-03 12:27:59.617205: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-03 12:27:59.692879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-03 12:27:59.693713: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-03 12:27:59.693764: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-03 12:27:59.921479: I tensorflow/str

#### 3) evaluate

In [ ]:
python ./keras_retinanet/bin/evaluate.py --backbone='resnet50' --iou-threshold 0.50 --image-min-side 600 --image-max-side 1000 csv val_annotations.csv classes.csv /snapshots/resnet50_csv_14.h5 --convert-model 